# Advanced Usage of reLAISS
### Authors: Evan Reynolds and Alex Gagliano

## Introduction

This notebook demonstrates advanced features of the reLAISS library for finding similar astronomical transients. While the basic_usage.ipynb notebook covered the fundamental functionality, here we'll explore more sophisticated techniques that give you greater flexibility and power in your analysis.

These advanced features allow you to customize how reLAISS processes and analyzes data, including dimensionality reduction, theorized lightcurves, host galaxy swapping, fine-tuning of algorithm parameters, visualization tools, and advanced anomaly detection.

## Topics Covered
1. Using PCA for dimensionality reduction
2. Creating and using theorized lightcurves
3. Swapping host galaxies
4. Setting maximum neighbor distances
5. Tweaking ANNOY parameters
6. Making corner plots
7. Advanced anomaly detection with parameter tuning
8. Host swapping in anomaly detection

## Setup

First, let's import the necessary packages and create the required directories:

In [ ]:
import os
import pandas as pd
import numpy as np
import relaiss as rl
import astropy.units as u
import matplotlib.pyplot as plt

# Create output directories
os.makedirs('./figures', exist_ok=True)
os.makedirs('./sfddata-master', exist_ok=True)
os.makedirs('./models', exist_ok=True)
os.makedirs('./timeseries', exist_ok=True)

def create_theorized_lightcurve():
    """Create a simple theorized lightcurve for demonstration."""
    # Create time points
    times = np.linspace(0, 100, 50) * u.day
    # Create magnitudes (simple gaussian)
    mags = 20 + 2 * np.exp(-(times.value - 50)**2 / 100)
    # Create errors
    errors = np.ones_like(mags) * 0.1
    
    # Create DataFrame in ANTARES format
    df = pd.DataFrame({
        'ant_mjd': times.to(u.day).value,
        'ant_mag': mags,
        'ant_magerr': errors,
        'ant_passband': ['g' if i % 2 == 0 else 'R' for i in range(len(times))]
    })
    return df

## Initialize the ReLAISS Client

We'll start by creating a ReLAISS client instance:

In [ ]:
# Initialize the client
client = rl.ReLAISS()

## 1. Using PCA for Dimensionality Reduction

PCA (Principal Component Analysis) can be used to reduce the dimensionality of the feature space while preserving most of the variance. This has several benefits:

- Improves search speed by reducing the computational complexity
- Potentially reduces noise in the feature space
- Helps mitigate the "curse of dimensionality" for high-dimensional data

To use PCA, we set `use_pca=True` in the `load_reference` method and specify the number of components to keep:

In [ ]:
client.load_reference(
    path_to_sfd_folder='./sfddata-master',
    use_pca=True,
    num_pca_components=20,  # Keep 20 PCA components
)

neighbors_df = client.find_neighbors(
    ztf_object_id='ZTF21abbzjeq',  # Using the test transient
    n=5,
    plot=True,
    save_figures=True,
    path_to_figure_directory='./figures'
)
print("\nNearest neighbors using PCA:")
print(neighbors_df)

## 2. Creating and Using Theorized Lightcurves

One powerful feature of reLAISS is the ability to use theorized (synthetic) lightcurves in the neighbor search. This allows you to:

- Test theoretical models against observed data
- Explore "what-if" scenarios by creating custom lightcurves
- Find real transients that match your theoretical predictions

Below, we create a simple Gaussian-shaped lightcurve and find its nearest neighbors:

In [ ]:
# Create a theorized lightcurve
theorized_lc = create_theorized_lightcurve()

# Find neighbors for the theorized lightcurve
# Need to provide a host galaxy when using theorized lightcurve
neighbors_df = client.find_neighbors(
    theorized_lightcurve_df=theorized_lc,
    host_ztf_id='ZTF21abbzjeq',  # Use this transient's host
    n=5,
    plot=True,
    save_figures=True,
    path_to_figure_directory='./figures'
)
print("\nNearest neighbors for theorized lightcurve:")
print(neighbors_df)

## 3. Swapping Host Galaxies

reLAISS allows you to swap the host galaxy of a transient, which is useful for:

- Exploring how host properties affect the similarity search results
- Investigating the effects of different environments on transient characteristics
- Testing hypotheses about host galaxy influences

Here's how to swap in a different host galaxy:

In [ ]:
# Find neighbors with a swapped host galaxy
neighbors_df = client.find_neighbors(
    ztf_object_id='ZTF21abbzjeq',  # Source transient
    host_ztf_id='ZTF20aacbyec',  # Host to swap in
    n=5,
    plot=True,
    save_figures=True,
    path_to_figure_directory='./figures'
)
print("\nNearest neighbors with swapped host galaxy:")
print(neighbors_df)

## 4. Setting Maximum Neighbor Distances

Sometimes you're only interested in neighbors that are truly similar to your target. By setting a maximum distance threshold, you can:

- Filter out neighbors that are too dissimilar
- Focus only on highly confident matches
- Avoid including poor matches just to reach a specific number of neighbors

Note that you might get fewer neighbors than requested if the distance threshold is applied:

In [ ]:
# Find neighbors with maximum distance constraint
neighbors_df = client.find_neighbors(
    ztf_object_id='ZTF21abbzjeq',
    n=5,
    max_neighbor_dist=5.0,  # Only return neighbors within this distance
    plot=True,
    save_figures=True,
    path_to_figure_directory='./figures'
)
print("\nNearest neighbors with maximum distance constraint:")
print(neighbors_df)
print(f"Number of neighbors found: {len(neighbors_df)}")

## 5. Tweaking ANNOY Parameters

ANNOY (Approximate Nearest Neighbors Oh Yeah) is the algorithm used for fast nearest neighbor search. You can tune its parameters to balance search accuracy and speed:

- `search_k`: Controls the number of nodes to explore during search (higher = more accurate but slower)
- `n_trees`: Controls the number of random projection trees built (set during client initialization)

Here's how to adjust the search_k parameter:

In [ ]:
# Find neighbors with tweaked ANNOY parameters
neighbors_df = client.find_neighbors(
    ztf_object_id='ZTF21abbzjeq',
    n=5,
    search_k=2000,  # Increase search_k for more accurate results
    plot=True,
    save_figures=True,
    path_to_figure_directory='./figures'
)
print("\nNearest neighbors with tweaked ANNOY parameters:")
print(neighbors_df)

## 6. Making Corner Plots

Corner plots are a powerful visualization tool that show the distribution of features for the input transient and its neighbors. They can help you:

- Understand which features are driving the similarity matching
- Identify potential correlations between different features
- Visualize the feature space and where your transient sits within it

To create corner plots, we need to first get the primer_dict containing information about the input transient:

In [ ]:
# Get neighbors from a new search
neighbors_df = client.find_neighbors(
    ztf_object_id='ZTF21abbzjeq',
    n=5,
    plot=True,
    save_figures=True,
    path_to_figure_directory='./figures'
)

# Get primer_dict separately
from relaiss.search import primer
primer_dict = primer(
    lc_ztf_id='ZTF21abbzjeq',
    theorized_lightcurve_df=None,
    host_ztf_id=None,
    dataset_bank_path=client.bank_csv,
    path_to_timeseries_folder='./',
    path_to_sfd_folder=client.path_to_sfd_folder,
    lc_features=client.lc_features,
    host_features=client.host_features,
    num_sims=0,
    save_timeseries=False,
)

# Create corner plots using the primer_dict
from relaiss.plotting import corner_plot
corner_plot(
    neighbors_df=neighbors_df,
    primer_dict=primer_dict,
    path_to_dataset_bank=client.bank_csv,
    path_to_figure_directory='./figures',
    save_plots=True
)
print("Corner plots saved to ./figures/")

## 7. Advanced Anomaly Detection with Parameter Tuning

The anomaly detection module in reLAISS uses an Isolation Forest algorithm that can be tuned for different scenarios. Key parameters include:

- `n_estimators`: Number of base estimators (trees) in the ensemble
- `contamination`: Expected proportion of outliers in the dataset
- `max_samples`: Number of samples drawn to train each base estimator

Let's explore how different parameters affect the model's performance:

In [ ]:
from relaiss.anomaly import train_AD_model, anomaly_detection

# Train models with different parameters to compare
print("Training anomaly detection model with default parameters...")
default_model_path = train_AD_model(
    lc_features=client.lc_features,
    host_features=client.host_features,
    path_to_dataset_bank=client.bank_csv,
    path_to_sfd_folder='./sfddata-master',
    path_to_models_directory="./models",
    n_estimators=100,
    contamination=0.02,
    max_samples=256,
    force_retrain=True
)

print("Training anomaly detection model with more trees...")
model_more_trees_path = train_AD_model(
    lc_features=client.lc_features,
    host_features=client.host_features,
    path_to_dataset_bank=client.bank_csv,
    path_to_sfd_folder='./sfddata-master',
    path_to_models_directory="./models",
    n_estimators=200,  # More trees
    contamination=0.02,
    max_samples=256,
    force_retrain=True
)

print("Training anomaly detection model with higher contamination...")
model_higher_contam_path = train_AD_model(
    lc_features=client.lc_features,
    host_features=client.host_features,
    path_to_dataset_bank=client.bank_csv,
    path_to_sfd_folder='./sfddata-master',
    path_to_models_directory="./models",
    n_estimators=100,
    contamination=0.05,  # Higher contamination
    max_samples=256,
    force_retrain=True
)

# Run anomaly detection with default model
print("\nRunning anomaly detection with default model...")
anomaly_detection(
    transient_ztf_id="ZTF21abbzjeq",
    lc_features=client.lc_features,
    host_features=client.host_features,
    path_to_timeseries_folder="./timeseries",
    path_to_sfd_folder='./sfddata-master',
    path_to_dataset_bank=client.bank_csv,
    path_to_models_directory="./models",
    path_to_figure_directory="./figures/AD_default",
    save_figures=True,
    n_estimators=100,
    contamination=0.02,
    max_samples=256,
    force_retrain=False
)

## 8. Anomaly Detection with Host Swapping

Just as with neighbor searches, you can swap host galaxies for anomaly detection. This helps you understand how host properties contribute to a transient's anomaly score.

This feature is particularly useful for:
- Testing if the anomalous nature of a transient is due to its host galaxy
- Exploring the "what if" scenario of a transient occurring in a different environment
- Separating intrinsic transient anomalies from host-related factors

In [ ]:
# Use the default model but swap in a different host galaxy
anomaly_detection(
    transient_ztf_id="ZTF21abbzjeq",
    lc_features=client.lc_features,
    host_features=client.host_features,
    path_to_timeseries_folder="./timeseries",
    path_to_sfd_folder='./sfddata-master',
    path_to_dataset_bank=client.bank_csv,
    host_ztf_id_to_swap_in="ZTF20aacbyec",  # Swap in a different host
    path_to_models_directory="./models",
    path_to_figure_directory="./figures/AD_host_swap",
    save_figures=True,
    n_estimators=100,
    contamination=0.02,
    max_samples=256,
    force_retrain=False
)

print("Anomaly detection figures saved to ./figures/AD_default/ and ./figures/AD_host_swap/")

## Conclusion

By combining these features, you can create highly customized searches tailored to your specific research questions.

For information on how to build your own dataset bank for reLAISS, see the `build_databank.ipynb` notebook.